In [ ]:
#create word2vec word embedding for reps using data already cleaned for glove

In [1]:
from __future__ import division
import random
import re
import nltk
import string
import gc
import numpy as np
import pandas as pd
from collections import Counter
import operator
import random
from gensim.models import Word2Vec

In [2]:
INPUT = "/home/twalton_umass_edu/Political Polarization Project/tmls/month_tmls_nrt/"
OUTPUT = "/home/twalton_umass_edu/Political Polarization Project/tmls/word_embeddings/"
months = ["jun", "jul", "aug", "sep", "oct", "nov", "dec", "jan"]
RNG = random.Random()
RNG.seed(400)

In [4]:
#function for cleaning text
#set the stemmer
sno = nltk.stem.SnowballStemmer('english')

#list of punctuation to be removed
punct_chars = list((set(string.punctuation) | {'’', '‘', '–', '—', '~', '|', '“', '”', '…', "'", "`", '_'}) - set(['#']))
#sort the punctuation
punct_chars.sort()
#make puncutation one string
punctuation = ''.join(punct_chars)
#symbols to be removed
replace = re.compile('[%s]' % re.escape(punctuation))

##########################################
######function for cleaning text##########
##########################################
def clean_text(text, event=None, stem=True):
    #remove emojis
    text = re.sub('<U\+[^>]+>', '', text)
    #replace &amp; with and
    text = re.sub('&amp;', 'and', text)
    # lower case
    text = text.lower()
    # eliminate urls
    text = re.sub(r'http\S*|\S*\.com\S*|\S*www\S*', ' ', text)
    #eliminate @mentions
    text = re.sub(r'\s@\S+', ' ', text)
    # substitute all other punctuation with whitespace
    text = replace.sub(' ', text)
    # replace all whitespace with a single space
    text = re.sub(r'\s+', ' ', text)
    # strip off spaces on either end
    text = text.strip()
    # stem words
    words = text.split()
    if stem:
        words = [sno.stem(w) for w in words]
    return words

In [5]:
#function for cleaning tweet and keeping words in vocab set
def clean_tweet(text):
    cleaned = clean_text(text)
    return ' '.join([w for w in cleaned if w in vocab])

In [6]:
#takes df of all samples and returns vocab set
def get_vocab(df):
    word_counts = Counter(clean_text(' '.join(df['text'])))
    vocab = []
    for k, v in word_counts.items():
        if v >= 10:
            vocab.append(k)
    return set(vocab)

In [7]:
#define function for getting samples for developing a vocab
def get_sample(df):
    print('sample')
    #get sample of 30000
    return df.sample(min(len(df), 30000), random_state = 10)

In [8]:
#load the monthly data and filter to only republicans
list_dfs = []

for m in months:
    df = pd.read_csv(INPUT + m + '_all_nrt_tweets.csv', encoding = 'UTF-8', dtype = 'str')
    list_dfs.append(df[(df['dem_follows'] == '0') & (df['rep_follows'] == '1')]) #only append rep data
    
#print number of tweets in each dataframe
for i, df in enumerate(list_dfs):
    print(len(df))

1521899
1471723
1562853
1496155
1653814
1618817
1267603
850552


In [9]:
#keep only english tweets and remove the proportion of tweets removed for each month
for i, df in enumerate(list_dfs):
    #print proportion that is not english
    print(len(df[df['lang'] != "en"]) / len(df))
    #keep only english tweets in the list of dfs
    list_dfs[i] = df[df['lang'] == "en"]

0.11507728173814426
0.1185882125916358
0.12101649995233077
0.12096273447604025
0.13486764533375578
0.14867832497434855
0.14066075892846577
0.12019841232517235


In [10]:
#get vocabs

#get montly samples, which are used to determine vocab
samples_list = []

for i, df in enumerate(list_dfs):
    samples_list.append(get_sample(df))
    
#convert into a single dataframe
samples = pd.concat(samples_list, ignore_index = True)
del samples_list

#get vocab
vocab = get_vocab(samples)
print(len(vocab))

#save vocab
with open(OUTPUT + 'rep_clean/joint_vocab.txt', 'w') as f:
    f.write('\n'.join(vocab))

sample
sample
sample
sample
sample
sample
sample
sample
12612


In [11]:
#loop through dfs and clean text
for i, df in enumerate(list_dfs):
    list_dfs[i]['text'] = list_dfs[i]['text'].astype(str).apply(clean_tweet)
    print('before cleaning :' + str(len(list_dfs[i])))
    #drop tweets that are whitespace
    list_dfs[i] = df[df['text'].str.contains(' ')]
    print('after cleaning :' + str(len(list_dfs[i])))

before cleaning :1346763
after cleaning :1320012
before cleaning :1297194
after cleaning :1271015
before cleaning :1373722
after cleaning :1346811
before cleaning :1315176
after cleaning :1288970
before cleaning :1430768
after cleaning :1398755
before cleaning :1378134
after cleaning :1339435
before cleaning :1089301
after cleaning :1061463
before cleaning :748317
after cleaning :732420


In [12]:
#convert into a single data frame
df = pd.concat(list_dfs, ignore_index = True)

#write the cleaned text
with open(OUTPUT + 'rep_clean/cleaned_text.txt', 'w') as f:
    f.write('\n'.join(df['text']))

In [13]:
#convert to a list of lists for feeding into the word2vec model
final = [tweet.split() for tweet in df['text'].tolist()]

In [15]:
del df

In [16]:
model = Word2Vec(sentences=final, size=100, window=5, min_count=1, workers=12)

model.save(OUTPUT + "rep_word2vec_100.model")